In [1]:
import warnings
warnings.filterwarnings('ignore')
from glob import glob
dog_filepaths = glob('data/dog/*.jpg')

['data/dog/294.jpg',
 'data/dog/4.jpg',
 'data/dog/430.jpg',
 'data/dog/194.jpg',
 'data/dog/321.jpg',
 'data/dog/399.jpg',
 'data/dog/130.jpg',
 'data/dog/86.jpg',
 'data/dog/24.jpg',
 'data/dog/318.jpg',
 'data/dog/213.jpg',
 'data/dog/154.jpg',
 'data/dog/38.jpg',
 'data/dog/408.jpg',
 'data/dog/393.jpg',
 'data/dog/88.jpg',
 'data/dog/403.jpg',
 'data/dog/37.jpg',
 'data/dog/146.jpg',
 'data/dog/91.jpg',
 'data/dog/332.jpg',
 'data/dog/172.jpg',
 'data/dog/220.jpg',
 'data/dog/54.jpg',
 'data/dog/3.jpg',
 'data/dog/129.jpg',
 'data/dog/292.jpg',
 'data/dog/188.jpg',
 'data/dog/142.jpg',
 'data/dog/17.jpg',
 'data/dog/80.jpg',
 'data/dog/439.jpg',
 'data/dog/244.jpg',
 'data/dog/296.jpg',
 'data/dog/68.jpg',
 'data/dog/368.jpg',
 'data/dog/41.jpg',
 'data/dog/83.jpg',
 'data/dog/364.jpg',
 'data/dog/73.jpg',
 'data/dog/30.jpg',
 'data/dog/211.jpg',
 'data/dog/69.jpg',
 'data/dog/20.jpg',
 'data/dog/39.jpg',
 'data/dog/149.jpg',
 'data/dog/1.jpg',
 'data/dog/208.jpg',
 'data/dog/58.j

In [2]:
cat_filepaths = glob('data/cat/*.jpg')

['data/cat/135.jpg',
 'data/cat/262.jpg',
 'data/cat/4.jpg',
 'data/cat/74.jpg',
 'data/cat/130.jpg',
 'data/cat/86.jpg',
 'data/cat/24.jpg',
 'data/cat/248.jpg',
 'data/cat/62.jpg',
 'data/cat/213.jpg',
 'data/cat/154.jpg',
 'data/cat/261.jpg',
 'data/cat/121.jpg',
 'data/cat/38.jpg',
 'data/cat/311.jpg',
 'data/cat/99.jpg',
 'data/cat/88.jpg',
 'data/cat/37.jpg',
 'data/cat/228.jpg',
 'data/cat/146.jpg',
 'data/cat/284.jpg',
 'data/cat/91.jpg',
 'data/cat/263.jpg',
 'data/cat/220.jpg',
 'data/cat/54.jpg',
 'data/cat/118.jpg',
 'data/cat/3.jpg',
 'data/cat/129.jpg',
 'data/cat/120.jpg',
 'data/cat/188.jpg',
 'data/cat/142.jpg',
 'data/cat/17.jpg',
 'data/cat/80.jpg',
 'data/cat/296.jpg',
 'data/cat/312.jpg',
 'data/cat/68.jpg',
 'data/cat/41.jpg',
 'data/cat/108.jpg',
 'data/cat/81.jpg',
 'data/cat/83.jpg',
 'data/cat/278.jpg',
 'data/cat/73.jpg',
 'data/cat/297.jpg',
 'data/cat/30.jpg',
 'data/cat/211.jpg',
 'data/cat/69.jpg',
 'data/cat/96.jpg',
 'data/cat/20.jpg',
 'data/cat/39.jpg

In [3]:
from PIL import Image
import numpy as np
import chainer

In [4]:
imgArray = []
x, t = [], []
dog_t = np.array(0, 'i')
cat_t = np.array(1, 'i')

for dog_filepath, cat_filepath in zip(dog_filepaths, cat_filepaths):
    dog_img = Image.open(dog_filepath)
    cat_img = Image.open(cat_filepath)
    dog_img = chainer.links.model.vision.vgg.prepare(dog_img, size=(224, 224))
    cat_img = chainer.links.model.vision.vgg.prepare(cat_img, size=(224, 224))
    x.append(dog_img)
    x.append(cat_img)
    t.append(dog_t)
    t.append(cat_t)

In [5]:
dataset = chainer.datasets.TupleDataset(x, t)
len(dataset)

500

In [6]:
import chainer
import chainer.links as L
import chainer.functions as F

In [8]:
class VGG16Model(chainer.Chain):

    def __init__(self, out_size):
        super(VGG16Model, self).__init__(
            base = L.VGG16Layers(),
            fc = L.Linear(None, out_size)
        )

    def __call__(self, x):
        h = self.base(x, layers=['fc7'])
        y = self.fc(h['fc7'])
        return y

In [9]:
import random

def reset_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    if chainer.cuda.available:
        chainer.cuda.cupy.random.seed(seed)

In [10]:
# CPUとGPU関連のシードをすべて固定
reset_seed(0)

In [25]:
# インスタンス化
model = L.Classifier(VGG16Model(out_size=2))
optimizer = chainer.optimizers.Adam(alpha=1e-4)
optimizer.setup(model)
model.predictor.base.disable_update()

In [26]:
gpu_id = 0  # 使用したGPUに割り振られているID
model.to_gpu(gpu_id)

In [27]:
batchsize = 32

train = int(len(dataset) * 0.7)
train, test = chainer.datasets.split_dataset_random(dataset, train, seed=1)

train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter = chainer.iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)

In [28]:
from chainer import training
from chainer.training import extensions

epoch = 10

updater = training.StandardUpdater(train_iter, optimizer, device=gpu_id)

trainer = training.Trainer(updater, (epoch, 'epoch'), out='kadai')

# バリデーション用のデータで評価
trainer.extend(extensions.Evaluator(test_iter, model, device=gpu_id))

# 学習結果の途中を表示する
trainer.extend(extensions.LogReport(trigger=(1, 'epoch')))

# １エポックごとに結果をlogファイルに出力させる
trainer.extend(extensions.PrintReport(['epoch', 'main/accuracy', 'validation/main/accuracy', 'main/loss', 'validation/main/loss', 'elapsed_time']), trigger=(1, 'epoch'))

In [29]:
trainer.run()

epoch       main/accuracy  validation/main/accuracy  main/loss   validation/main/loss  elapsed_time
1           0.551136       0.735795                  1.17374     0.606504              26.5468       
2           0.701705       0.832386                  0.696422    0.402964              53.1001       
3           0.832386       0.860227                  0.427926    0.305278              79.6882       
4           0.875          0.888068                  0.340569    0.249377              106.261       
5           0.880682       0.894318                  0.284275    0.215288              132.823       
6           0.917614       0.909659                  0.186529    0.192817              159.401       
7           0.923295       0.93125                   0.20438     0.174901              186.044       
8           0.934659       0.93125                   0.165779    0.163113              212.643       
9           0.940341       0.93125                   0.150363    0.153847           

In [30]:
import json
import pandas as pd

In [31]:
with open('kadai/log') as f:
    result = pd.DataFrame(json.load(f))

In [32]:
result

,elapsed_time,epoch,iteration,main/accuracy,main/loss,validation/main/accuracy,validation/main/loss
0,26.546767,1,11,0.551136,1.173742,0.735795,0.606504
1,53.100060,2,22,0.701705,0.696422,0.832386,0.402964
2,79.688246,3,33,0.832386,0.427926,0.860227,0.305278
3,106.261180,4,44,0.875000,0.340569,0.888068,0.249377
4,132.823147,5,55,0.880682,0.284275,0.894318,0.215288
5,159.400518,6,66,0.917614,0.186529,0.909659,0.192817
6,186.043786,7,77,0.923295,0.204380,0.931250,0.174901
7,212.642744,8,88,0.934659,0.165779,0.931250,0.163113
8,239.282478,9,99,0.940341,0.150363,0.931250,0.153847
9,265.890304,10,110,0.948864,0.121023,0.943750,0.146195


In [33]:
result[['main/loss', 'validation/main/loss']].plot()

In [34]:
result[['main/accuracy', 'validation/main/accuracy']].plot()